In [11]:
from dask.distributed import Client
import multiprocessing

def client():
    n_workers = multiprocessing.cpu_count()
    threads_per_worker = 2
    client = Client(n_workers=n_workers, threads_per_worker=threads_per_worker)
    return client

client = client()

/home/punitsureka/Desktop/talview/testing-dask/venv/lib/python3.10/site-packages/distributed/node.py:182: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 43489 instead
  warnings.warn(


In [2]:
import pandas as pd
import dask.dataframe as dd

In [3]:
top_tags_pandas = pd.read_csv('/home/punitsureka/Desktop/talview/testing-dask/y_top_10000_tags.csv')
top_tags = dd.from_pandas(top_tags_pandas, npartitions=100)
tags_2022_pandas = pd.read_csv('/home/punitsureka/Desktop/talview/testing-dask/stackoverflow_questions_2022.csv', usecols = ['tags'])
tags_2022 = dd.from_pandas(tags_2022_pandas, npartitions=100)
tags_list = tags_2022['tags'].str.split('|')
tags_list = tags_list.dropna()

In [7]:
def compute_top_10(tag_x):
    filtered_series = tags_list[tags_list.apply(lambda x: tag_x in x)]
    flattened_series = filtered_series.explode()
    tag_counts = flattened_series.value_counts()
    top_10 = tag_counts.nlargest(11)

    top_10_percentages = {}
    for tag, count in top_10.iteritems():
        if tag != tag_x:
            percentage = round(count / tag_counts[tag_x] * 100, 2)
            top_10_percentages[tag] = f"{percentage}%"

    return (tag_x, top_10_percentages)

In [12]:
results = client.map(compute_top_10, top_tags['tag_name'])